# Урок № 2
### [Картографические проекции](https://autogis-site.readthedocs.io/en/latest/notebooks/L2/02-projections.html)

**Координатные системы отсчета (CRS)** важны, поскольку геометрические фигуры в геодезическом каркасе-это просто набор координат в произвольном пространстве. CRS сообщает Python, как эти координаты связаны с местами на Земле. **Картографическая проекция** (или проекционная система координат) - это систематическое преобразование широты и долготы в плоскую поверхность, где единицы измерения довольно часто представлены в виде метров (вместо десятичных градусов). Это преобразование используется для представления трехмерной земли на плоской двумерной карте.

Поскольку CRS в разных пространственных наборах данных довольно часто различаются (например, у одного могут быть координаты, определенные в десятичных градусах, а у другого-в метрах), это обычная процедура перепроектирования (преобразования) различных слоев в общую CRS. Важно, чтобы слои находились в одной координатной системе отсчета при анализе пространственных отношений между слоями, например, при создании точки в полигональном запросе или другом типе анализа наложения.

Выбор подходящей проекции для вашей карты не всегда прост, потому что это зависит от того, что вы на самом деле хотите представить с помощью вашей карты, и каков пространственный масштаб ваших данных. На самом деле, нет ни одной “идеальной проекции”, так как каждая из них имеет свои сильные и слабые стороны, и вы должны выбрать проекцию, которая лучше всего соответствует вашим потребностям. На самом деле, проекция, которую вы выберете, может даже рассказать что-то о вас!

![](img/Map-projections.png)

Источник: XKCD, Смотрите полный комикс [о том,” что ваша любимая картографическая проекция говорит о вас"](https://xkcd.com/977/).

Для тех из вас, кто хочет немного более аналитического подхода к выбору проекции, вы можете получить хороший обзор от [georeference.org](http://www.georeference.org/doc/guide_to_selecting_map_projections.htm), или из этого сообщения в блоге, представляющего [сильные и слабые стороны нескольких широко используемых прогнозов.](http://usersguidetotheuniverse.com/index.php/2011/03/03/whats-the-best-map-projection/)



##### Система координат (CRS) в Geopandas

К счастью, определить и изменить проекции легко с помощью **Geopandas** и библиотеки под названием **pyproj**. В этом уроке мы увидим, как получить координируйте информацию о системе отсчета из этих данных и о том, как перепроектировать эти данные в другую crs. Мы перепроектируем файл данных из **WGS84** (координаты ``lat``, ``lon``) в азимутальную проекцию Ламберта равной площади, которая является рекомендуемой проекцией для Европа Европейской комиссией.

Для этого урока мы будем использовать Шейп-файл под названием Europe_borders.shprepresenting the country borders in Europe, который вы уже должны были загрузить во время предыдущего урока в L2_data папку.

В Шейп-файлах информация о системе координат хранится в файле .prj-file. Если этот файл отсутствует, у вас могут быть неприятности!. При считывании данных в GeoDataFrame с Geopandas CRS информация автоматически сохраняется в `.crs` атрибуте GeoDataFrame.

Давайте начнем с чтения данных из Europe_borders.shp файла и проверки crs:

In [2]:
import os
import geopandas as gpd


# Корректное отображение графиков в jupyter
%matplotlib inline

In [4]:
# Текущая директория
ROOT_PATH = os.path.abspath(os.curdir)

# Папка с данными
fp = os.path.join(ROOT_PATH, r"L2_data\Europe_borders.shp")

# Чтение
data = gpd.read_file(fp)
data.head()

,TZID,geometry
0,Europe/Berlin,"POLYGON ((8.45778 54.56236, 8.44953 54.56269, ..."
1,Europe/Berlin,"POLYGON ((8.71992 47.69664, 8.72092 47.69530, ..."
2,Europe/Berlin,"POLYGON ((6.73317 53.57409, 6.73017 53.57542, ..."
3,Europe/Berlin,"POLYGON ((6.85822 53.59411, 6.85592 53.59550, ..."
4,Europe/Berlin,"POLYGON ((6.89894 53.62561, 6.88439 53.62814, ..."


In [6]:
# Проверьте данные системы координат
data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

То, что мы видим здесь, на самом деле является объектом CRS из модуля pyproj.

Номер **EPSG** (названный в честь европейской группы нефтяных исследований) - это код, который сообщает о системе координат набора данных. "[Набор данных геодезических параметров EPSG](http://www.epsg.org/) представляет собой набор определений координатных систем отсчета и преобразований координат, которые могут быть глобальными, региональными, национальными или локальными в применении”.

Код **EPSG** нашего geodataframe `4326`- это код, который относится к системе координат **WGS84** (мы также можем выяснить это, посмотрев на значения координат, которые являются десятичными градусами долготы и широты).

Вы можете найти много информации и списков доступных систем координат из:

www.spatialreference.org

www.proj4.org

www.mapref.org

Давайте продолжим проверку значений в нашем geometry столбце -, чтобы убедиться, что CRS нашего GeoDataFrame кажется правильным:

In [7]:
data["geometry"]

0       POLYGON ((8.45778 54.56236, 8.44953 54.56269, ...
1       POLYGON ((8.71992 47.69664, 8.72092 47.69530, ...
2       POLYGON ((6.73317 53.57409, 6.73017 53.57542, ...
3       POLYGON ((6.85822 53.59411, 6.85592 53.59550, ...
4       POLYGON ((6.89894 53.62561, 6.88439 53.62814, ...
                              ...                        
3627    POLYGON ((18.75422 59.49353, 18.74311 59.49050...
3628    POLYGON ((18.84164 59.50605, 18.84028 59.50797...
3629    POLYGON ((18.59995 59.45653, 18.59431 59.45964...
3630    POLYGON ((30.00000 51.48947, 29.99511 51.48572...
3631    POLYGON ((-5.34392 36.11253, -5.34717 36.11528...
Name: geometry, Length: 3632, dtype: geometry

Как мы видим, значения координат полигонов действительно выглядят как значения широты и долготы, так что все вроде бы в порядке.

Проекция WGS84 на самом деле не очень хороша для представления европейских границ на карте (области искажаются), поэтому давайте преобразуем эти геометрии в проекцию Ламберта азимутальной равной площади [(EPSG: 3035)](http://spatialreference.org/ref/epsg/etrs89-etrs-laea/), которая является рекомендуемой проекцией Европейской комиссии.

Изменение проекции очень просто [сделать в Geopandas](http://geopandas.org/projections.html#re-projecting) с `.to_crs()` - function, которая является встроенной функцией GeoDataFrame. Функция имеет два альтернативных параметра 1) `crs` и 2) `epgs`, которые можно использовать для преобразования координат и повторного проецирования данных в CRS, который вы хотите использовать.

Давайте перепроектируем наши данные в EPSG 3035 используя epgs - параметр:

In [9]:
# делаем полную копию данных
data_WGS84 = data.copy()
data_WGS84

,TZID,geometry
0,Europe/Berlin,"POLYGON ((8.45778 54.56236, 8.44953 54.56269, ..."
1,Europe/Berlin,"POLYGON ((8.71992 47.69664, 8.72092 47.69530, ..."
2,Europe/Berlin,"POLYGON ((6.73317 53.57409, 6.73017 53.57542, ..."
3,Europe/Berlin,"POLYGON ((6.85822 53.59411, 6.85592 53.59550, ..."
4,Europe/Berlin,"POLYGON ((6.89894 53.62561, 6.88439 53.62814, ..."
...,...,...
3627,Europe/Stockholm,"POLYGON ((18.75422 59.49353, 18.74311 59.49050..."
3628,Europe/Stockholm,"POLYGON ((18.84164 59.50605, 18.84028 59.50797..."
3629,Europe/Stockholm,"POLYGON ((18.59995 59.45653, 18.59431 59.45964..."
3630,Europe/Minsk,"POLYGON ((30.00000 51.48947, 29.99511 51.48572..."


In [11]:
data = data.to_crs(epsg=3035)
data

,TZID,geometry
0,Europe/Berlin,"POLYGON ((4221214.558 3496203.404, 4220681.651..."
1,Europe/Berlin,"POLYGON ((4224860.478 2732279.320, 4224932.819..."
2,Europe/Berlin,"POLYGON ((4104652.176 3390034.953, 4104460.401..."
3,Europe/Berlin,"POLYGON ((4113025.664 3391895.756, 4112879.943..."
4,Europe/Berlin,"POLYGON ((4115871.228 3395282.099, 4114921.348..."
...,...,...
3627,Europe/Stockholm,"POLYGON ((4816590.301 4074234.333, 4816008.390..."
3628,Europe/Stockholm,"POLYGON ((4821327.245 4076229.945, 4821222.901..."
3629,Europe/Stockholm,"POLYGON ((4808429.473 4069082.454, 4808067.721..."
3630,Europe/Minsk,"POLYGON ((5690071.718 3342608.395, 5689859.929..."
